In [10]:
import warnings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  
warnings.filterwarnings("ignore")

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import psycopg
from psycopg.rows import dict_row

db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
pg_collection = os.getenv("PG_COLLECTION")


In [4]:
conninfo = f"host={db_host} port={db_port} dbname={db_name} user={db_user} password={db_password}"

In [5]:
with psycopg.connect(conninfo, autocommit=True) as conn:
    with conn.cursor(row_factory=dict_row) as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        cur.execute(f'''
            CREATE TABLE IF NOT EXISTS {pg_collection} (
                id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
                source TEXT,
                chunk_no INT,
                content TEXT,
                metadata JSONB,
                embedding VECTOR(1024)
            );
        ''')
        
        cur.execute(f'''
            DO $$
            BEGIN
                IF NOT EXISTS (
                  SELECT 1 FROM pg_indexes WHERE indexname = 'idx_{pg_collection}_embedding'
                ) THEN
                  EXECUTE 'CREATE INDEX idx_{pg_collection}_embedding ON {pg_collection} USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100)';
                END IF;
            END $$;
        ''')

print("pgvector ready to use.")

pgvector ready to use.


## Load Dataset

In [6]:
df = pd.read_csv('../data/supermarket_preprocessed.csv')

In [7]:
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,...,Time,Payment,cogs,gross margin percentage,gross income,Rating,Hour,Hour_label,Month,Weekday
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,...,1900-01-01 13:08:00,Ewallet,522.83,4.761905,26.1415,9.1,13,01 PM,1,Saturday
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,...,1900-01-01 10:29:00,Cash,76.40,4.761905,3.8200,9.6,10,10 AM,3,Friday
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,...,1900-01-01 13:23:00,Credit card,324.31,4.761905,16.2155,7.4,13,01 PM,3,Sunday
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,...,1900-01-01 20:33:00,Ewallet,465.76,4.761905,23.2880,8.4,20,08 PM,1,Sunday
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,...,1900-01-01 10:37:00,Ewallet,604.17,4.761905,30.2085,5.3,10,10 AM,2,Friday


## Prepare Text And Metadata

In [8]:
docs = []
for i, row in df.iterrows():
    texts = (
        f"Invoice {row['Invoice ID']} | Branch: {row['Branch']} ({row['City']}) | "
        f"Customer: {row['Customer type']} ({row['Gender']}) | "
        f"Product: {row['Product line']} | Quantity: {row['Quantity']} | "
        f"Unit Price: {row['Unit price']} | Sales: {row['Sales']} | "
        f"Payment: {row['Payment']} | Date: {row['Date']} {row['Time']} "
        f"| Rating: {row['Rating']}"
    )

    meta = {col: row[col] for col in df.columns}  
    docs.append({"content": texts, "metadata": meta})

print('Total Documents', len(docs))

Total Documents 1000


## Chunking Data

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = []

for doc in docs:
    pieces = splitter.split_text(doc['content'])
    for i, piece in enumerate(pieces):
        chunks.append({'content': piece, 'metadata': doc['metadata'], 'chunk_index': i})


print('Total chunks:', len(chunks))

Total chunks: 1000


In [11]:
chunks[0]

{'content': 'Invoice 750-67-8428 | Branch: Alex (Yangon) | Customer: Member (Female) | Product: Health and beauty | Quantity: 7 | Unit Price: 74.69 | Sales: 548.9715 | Payment: Ewallet | Date: 2019-01-05 1900-01-01 13:08:00 | Rating: 9.1',
 'metadata': {'Invoice ID': '750-67-8428',
  'Branch': 'Alex',
  'City': 'Yangon',
  'Customer type': 'Member',
  'Gender': 'Female',
  'Product line': 'Health and beauty',
  'Unit price': 74.69,
  'Quantity': 7,
  'Tax 5%': 26.1415,
  'Sales': 548.9715,
  'Date': '2019-01-05',
  'Time': '1900-01-01 13:08:00',
  'Payment': 'Ewallet',
  'cogs': 522.83,
  'gross margin percentage': 4.761904762,
  'gross income': 26.1415,
  'Rating': 9.1,
  'Hour': 13,
  'Hour_label': '01 PM',
  'Month': 1,
  'Weekday': 'Saturday'},
 'chunk_index': 0}

## Embedding And Saving To PGVector

In [12]:
embedding = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [13]:
from langchain_community.vectorstores import PGVector
from uuid import uuid4

connection = f"postgresql+psycopg://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

vectorstore = PGVector(
    embedding_function=embedding,
    collection_name=pg_collection,
    connection_string=connection,
    use_jsonb=True
)


In [14]:
texts_chunks = [c['content'] for c in chunks]
metas_chunks = [c['metadata'] for c in chunks]
ids = [str(uuid4()) for _ in chunks]

if texts_chunks:
    vectorstore.add_texts(
        texts=texts_chunks,
        metadatas=metas_chunks,
        ids=ids
    )
    print("Chunks added successfully.")
else:
    print("No valid chunks to add.")

Chunks added successfully.


## Create Retrieval QA Chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
import getpass

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=GROQ_API_KEY, model='llama-3.1-8b-instant')

prompt_template = """
You are an expert business assistant for a retail store owner.
Your goal is to help the admin or owner analyze sales data, invoices, and business performance.

You have access to the following sales and transaction information:
{context}

Admin query: {question}

Answer as a professional business analyst:
- Provide clear insights based on the available data.
- Mention key fields such as Invoice ID, Branch, City, Product line, Quantity, Sales, Tax, Payment method, Date, and Rating if relevant.
- If the query is about totals or summaries, calculate or explain trends (e.g., total sales, average rating, best-selling product line).
- If exact data is not available, suggest related information from the context.
- Keep the answer concise, accurate, and suitable for a business decision-maker.
"""
PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["context", "question"])

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda

def format_docs(docs):
    out = []
    for i, d in enumerate(docs):
        src = d.metadata.get('product_id', "")
        out.append(f"[{i}] {src}\n{d.page_content}")
    return "\n\n".join(out)

if llm:
    rag_chain = (
         {
            'context': retriever | RunnableLambda(format_docs),
            'question': RunnablePassthrough()
         } 
         | PROMPT
         | llm
    )
    print("RAG chain created successfully.")
else :
    rag_chain = None
    print("Failed to create RAG chain.")

RAG chain created successfully.


In [18]:
def ask_customer_service(question:str):
    if rag_chain:
        response = rag_chain.invoke(question)
        return response.content if hasattr(response, "content") else str(response)
    else:
        return "Sorry, I can't assist with that."

In [25]:
print(ask_customer_service("when Invoice 718-57-9773? buy and how much our gross sales"))

**Invoice 718-57-9773 Details:**

Based on the provided sales data, I've extracted the following information related to Invoice 718-57-9773:

- **Invoice ID:** 718-57-9773
- **Branch:** Giza (Naypyitaw)
- **Product Line:** Sports and travel
- **Quantity:** 10 units
- **Unit Price:** $49.33
- **Sales (Gross):** $517.965
- **Payment Method:** Credit card
- **Date:** 2019-02-03, 16:40:00
- **Rating:** 9.4/10

**Gross Sales:**
The gross sales for this invoice are $517.965.

**Key Insights:**
This transaction represents a significant sale in the Sports and travel product line, with a high rating of 9.4/10. The sales amount is substantial, suggesting a high-value customer engagement.
